In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use("seaborn")
%matplotlib inline

import warnings
warnings.simplefilter("ignore")
from binance.client import Client
import time
import statistics as stats
from datetime import datetime
import glob
import mlfinlab as ml
from talib import *

In [22]:
#path = r'C:\DRO\DCL_rawdata_files' # use your path
directory = '../data/30min/btc'
all_files = glob.glob(directory + "/*.csv")

In [23]:
all_files

['../data/30min/btc\\ADABTC.csv',
 '../data/30min/btc\\BNBBTC.csv',
 '../data/30min/btc\\ETHBTC.csv',
 '../data/30min/btc\\LINKBTC.csv',
 '../data/30min/btc\\LTCBTC.csv',
 '../data/30min/btc\\XRPBTC.csv']

In [24]:
df_dict = {}

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df = df.drop(columns=['ret','trgt', 'Unnamed: 0'])
    df['bin'] = df['bin'].fillna(-5)
    day_times = []
    
    df['Close_pct'] = df['Close'].pct_change()
    df['Open_pct'] = df['Open'].pct_change()
    df['High_pct'] = df['High'].pct_change()
    df['Low_pct'] = df['Low'].pct_change()
    
    for i in df['Open Time']:
        time_of_day = i.split(' ')[1].split('.')[0].split(':')
        day_time = int(time_of_day[0]) * 3600 + int(time_of_day[1]) * 60 + int(time_of_day[2])
        day_times.append(day_time)
        
    df.insert(1, 'Time of Day', day_times)
    df_dict[filename.split('\\')[1][:-7]] = df
    
coin_list = list(df_dict.keys())

In [25]:
df_dict['ADA'].head()

,Open Time,Time of Day,Open,High,Low,Close,Volume,Close Time,Quote asset vol,Num trades,Taker buy base asset volume,Taker buy quote asset volume,Status,bin,Close_pct,Open_pct,High_pct,Low_pct
0,2017-11-30 12:00:00.000,43200,0.000009,0.000009,0.000009,0.000009,1064.0,2017-11-30 12:29:59.999,0.009576,1.0,0.0,0.000000,2.978003e+06,-5.0,NaN,NaN,NaN,NaN
1,2017-11-30 12:30:00.000,45000,0.000089,0.000089,0.000011,0.000012,5467734.0,2017-11-30 12:59:59.999,70.897764,757.0,2932866.0,38.860513,1.488462e+07,-5.0,0.366667,8.888889,8.888889,0.222222
2,2017-11-30 13:00:00.000,46800,0.000012,0.000012,0.000011,0.000012,2311453.0,2017-11-30 13:29:59.999,26.985643,494.0,1020028.0,11.869097,2.100100e+07,-5.0,-0.049593,-0.861798,-0.861685,0.006364
3,2017-11-30 13:30:00.000,48600,0.000012,0.000012,0.000011,0.000011,2394767.0,2017-11-30 13:59:59.999,27.390454,364.0,1007572.0,11.544766,2.447178e+07,-5.0,-0.029085,-0.061789,-0.049553,0.008130
4,2017-11-30 14:00:00.000,50400,0.000011,0.000012,0.000011,0.000012,2837921.0,2017-11-30 14:29:59.999,33.418909,365.0,1801604.0,21.206073,2.615221e+07,-5.0,0.064317,-0.016464,0.046154,0.016129


In [26]:
directory = '../data/indicators_30min/btc'
all_files = glob.glob(directory + "/*.csv")

In [27]:
sub_df_dict = {}

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    sub_df_dict[filename.split('\\')[1][:-7]] = df

In [28]:
sub_df_dict.keys()

dict_keys(['ADA', 'BNB', 'ETH', 'LINK', 'LTC', 'XRP'])

In [29]:
columns = list(sub_df_dict['ADA'].columns)
indicators = columns[columns.index('SMA'):]

In [30]:
for coin in coin_list:
    close = sub_df_dict[coin]['Close']
    upperband, middleband, lowerband = BBANDS(close, timeperiod=25, nbdevup=2, nbdevdn=2, matype=0)
    
    sub_df_dict[coin]['BB_upper'] = upperband
    sub_df_dict[coin]['BB_ema'] = middleband
    sub_df_dict[coin]['BB_lower'] = lowerband

In [31]:
sub_df_dict['ADA'].head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote asset vol,Num trades,Taker buy base asset volume,...,fastd,SAR,ULTOSC,WILLR,AD,ADOSC,OBV,ATR,NATR,Spinning Top
0,2017-11-30 12:00:00.000,0.000009,0.000009,0.000009,0.000009,1064.0,2017-11-30 12:29:59.999,0.009576,1.0,0.0,...,NaN,NaN,NaN,NaN,0.000000e+00,NaN,1064.0,NaN,NaN,0.0
1,2017-11-30 12:30:00.000,0.000089,0.000089,0.000011,0.000012,5467734.0,2017-11-30 12:59:59.999,70.897764,757.0,2932866.0,...,NaN,0.000009,NaN,NaN,-5.285476e+06,NaN,5468798.0,NaN,NaN,0.0
2,2017-11-30 13:00:00.000,0.000012,0.000012,0.000011,0.000012,2311453.0,2017-11-30 13:29:59.999,26.985643,494.0,1020028.0,...,NaN,0.000009,NaN,NaN,-5.285476e+06,NaN,3157345.0,NaN,NaN,0.0
3,2017-11-30 13:30:00.000,0.000012,0.000012,0.000011,0.000011,2394767.0,2017-11-30 13:59:59.999,27.390454,364.0,1007572.0,...,NaN,0.000009,NaN,NaN,-5.995037e+06,NaN,762578.0,NaN,NaN,0.0
4,2017-11-30 14:00:00.000,0.000011,0.000012,0.000011,0.000012,2837921.0,2017-11-30 14:29:59.999,33.418909,365.0,1801604.0,...,NaN,0.000009,NaN,NaN,-4.166154e+06,NaN,3600499.0,NaN,NaN,0.0


In [32]:
for coin in coin_list:
    df_dict[coin][indicators] = sub_df_dict[coin][indicators]
    df_dict[coin].dropna().to_csv(coin+'_final.csv')

In [241]:
coin_list

['ADA', 'BNB', 'ETH', 'LINK', 'LTC', 'XRP']

In [15]:
'''
from info_gain import info_gain

# Example of color to indicate whether something is fruit or vegatable
produce = ['apple', 'apple', 'apple', 'strawberry', 'eggplant']
fruit   = [ True  ,  True  ,  True  ,  True       ,  False    ]
colour  = ['green', 'green', 'red'  , 'red'       , 'purple'  ]

ig  = info_gain.info_gain(fruit, colour)
iv  = info_gain.intrinsic_value(fruit, colour)
igr = info_gain.info_gain_ratio(fruit, colour)

print(ig, iv, igr)
'''

0.5004024235381879 1.5219280948873621 0.3287950496604918


In [ ]:
'''
label based on:
bb
rsi
'''

In [7]:
dataframes = []
sub_columns = ['Open Time', 'Close Time', 'Open', 'High', 'Low', 'Close', 'Volume']

for df in li:
    dataframes.append(df[sub_columns])

In [8]:
dataframes[1].head()

,Open Time,Close Time,Open,High,Low,Close,Volume
0,2017-07-14 04:00:00.000,2017-07-14 04:29:59.999,0.000050,0.000053,0.000010,0.000047,4407125.0
1,2017-07-14 04:30:00.000,2017-07-14 04:59:59.999,0.000047,0.000055,0.000047,0.000050,2446523.0
2,2017-07-14 05:00:00.000,2017-07-14 05:29:59.999,0.000049,0.000049,0.000039,0.000044,1244735.0
3,2017-07-14 05:30:00.000,2017-07-14 05:59:59.999,0.000044,0.000046,0.000042,0.000044,490063.0
4,2017-07-14 06:00:00.000,2017-07-14 06:29:59.999,0.000044,0.000049,0.000044,0.000048,512689.0


In [95]:
def basic_labels(data, seq_len=11):
    
    labels = [0] * int((seq_len-1)/2)
    
    for i in range(len(data)):
        print(labels)
        
        if data[i:i+seq_len][int((seq_len-1)/2)] == max(data[i:i+seq_len]):
            #ret_list = [0]*
            labels.append(-1)
            
        elif data[i:i+seq_len][int((seq_len-1)/2)] == min(data[i:i+seq_len]):
            #ret_list = [0]
            labels.append(1)
       
        else:
            labels.append(0)
            
        return labels

In [9]:
def stupid_labels(data):
    
    data.loc[0]['label'] = 0
    data.loc[data.shape[0]-1]['label'] = 0
    
    for i in range(1, data.shape[0]-1):
        if list(data['Close'])[i - 1] > list(data['Close'])[i] and list(data['Close'])[i] < list(data['Close'])[i + 1]:
            data.loc[i]['label'] = 1
        elif list(data['Close'])[i - 1] < list(data['Close'])[i] and list(data['Close'])[i] > list(data['Close'])[i + 1]:
            data.loc[i]['label'] = -1
        else:
            data.loc[i]['label'] = 0